# 2. Užduotis

Tarkime, kad laikas iki įvykio $T$ turi ekstremalių reikšmių (minimalių) skirstinį, t.y.
pasiskirstymo funkcija

$$
$$

Naudodami atvirkštinės transformacijos metodą, sumodeliuokite paprastąją atsitiktinę imtį
(**Pastaba.** Kai iš konteksto aišku, dažnai žodis realizacija praleidžiamas).

Raskite taškinius ir intervalinius parametrų įverčius. Pasirinktame taške $t$ įvertinkite pasi-
skirstymo funkciją ir jos pasikliovimo intervalą.